In [ ]:
import gc
import os
import random
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import pandas as pd
import re

import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
with open('/content/drive/MyDrive/.pkl', 'rb') as f:
    group = pickle.load(f)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(0)

In [ ]:
from sklearn.model_selection import train_test_split
train_group, valid_group = train_test_split(group, test_size=0.1)
train_group.shape, valid_group.shape

((26143,), (2905,))

In [ ]:
class SAINTDataset(Dataset):
    def __init__(self, user_sequences, num_questions, subset='train', max_seq=100, min_seq=10):
        super(SAINTDataset, self).__init__()
        self.max_seq = max_seq
        self.num_questions = num_questions
        self.user_sequences = user_sequences
        self.subset = subset

        self.user_ids = []
        for user_id in user_sequences.index:
            q, _, _ = user_sequences[user_id]
            # if len(q) < min_seq:
            #     continue
            self.user_ids.append(user_id)

    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, index):
        user_id = self.user_ids[index]

        q_, qa_, diff_ = self.user_sequences[user_id]
        seq_len = len(q_)

        q = np.zeros(self.max_seq, dtype=int)
        qa = np.zeros(self.max_seq, dtype=int)
        diff = np.zeros(self.max_seq, dtype=int)


        # If there are more questions answered than max_seq, take the last max_seq sequences
        if seq_len >= self.max_seq:
            q[:] = q_[-self.max_seq:]
            qa[:] = qa_[-self.max_seq:]
            diff[:] = diff_[-self.max_seq:]

        # If not, map our user_sequences to the tail end of q and qa, the start will be padded with zeros
        else:
            q[-seq_len:] = q_
            qa[-seq_len:] = qa_
            diff[-seq_len:] = diff_


        r = np.zeros(self.max_seq, dtype=int)   #shifted qa
        r[1:] = qa[:-1].copy()

        return q, r, qa, diff

### 인코더에 diff 추가

In [ ]:
class FFN(nn.Module):
    def __init__(self, dim=128):
        super().__init__()
        self.layer1 = nn.Linear(dim, dim)
        self.layer2 = nn.Linear(dim, dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.layer2(   self.relu(   self.layer1(x)))


def future_mask(seq_length):    #다음 시점 마스킹
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)


class Encoder(nn.Module):
    def __init__(self, n_in, seq_len=100, embed_dim=128, nheads=4):
        super().__init__()

        #e : q seq
        #n_in: total num of input

        self.seq_len = seq_len
        self.diff_embed = nn.Embedding(NUM_DIFFS, embed_dim)
        #self.part_embed = nn.Embedding(10, embed_dim)

        self.e_embed = nn.Embedding(n_in, embed_dim)
        self.e_pos_embed = nn.Embedding(seq_len, embed_dim)
        self.e_norm = nn.LayerNorm(embed_dim)

        self.e_multi_att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=nheads, dropout=0.2)
        self.m_norm = nn.LayerNorm(embed_dim)
        self.ffn = FFN(embed_dim)

    def forward(self, e,diff, first_block=True):

        if first_block:
            e = self.e_embed(e)
            diff = self.diff_embed(diff)
            e = e+ diff

        pos = torch.arange(self.seq_len).unsqueeze(0).to(device)
        e_pos = self.e_pos_embed(pos)
        e = e + e_pos
        e = self.e_norm(e)
        e = e.permute(1,0,2) #[bs, s_len, embed] => [s_len, bs, embed]
        n = e.shape[0]

        att_mask = future_mask(n).to(device)
        att_out, _ = self.e_multi_att(e, e, e, attn_mask=att_mask)
        m = e + att_out
        m = m.permute(1,0,2)

        o = m + self.ffn(self.m_norm(m))

        return o

class Decoder(nn.Module):
    def __init__(self, n_in, seq_len=100, embed_dim=128, nheads=4):
        super().__init__()
        self.seq_len = seq_len

        self.r_embed = nn.Embedding(n_in, embed_dim)    #r: 이전 시점 정답여부
        self.r_pos_embed = nn.Embedding(seq_len, embed_dim)
        self.r_norm = nn.LayerNorm(embed_dim)
        #self.diff_embed = nn.Embedding(NUM_DIFFS, embed_dim)


        self.r_multi_att1 = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=4, dropout=0.2)
        self.r_multi_att2 = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=4, dropout=0.2)
        self.ffn = FFN(embed_dim)

        self.r_norm1 = nn.LayerNorm(embed_dim)
        self.r_norm2 = nn.LayerNorm(embed_dim)
        self.r_norm3 = nn.LayerNorm(embed_dim)


    def forward(self, r, o,  first_block=True):

        if first_block:
            r = self.r_embed(r)
            #diff = self.diff_embed(diff)


            #r = r + diff

        pos = torch.arange(self.seq_len).unsqueeze(0).to(device)
        r_pos_embed = self.r_pos_embed(pos)
        r = r + r_pos_embed
        r = self.r_norm1(r)
        r = r.permute(1,0,2)
        n = r.shape[0]

        att_out1, _ = self.r_multi_att1(r, r, r, attn_mask=future_mask(n).to(device))
        m1 = r + att_out1

        o = o.permute(1,0,2)
        o = self.r_norm2(o)
        att_out2, _ = self.r_multi_att2(m1, o, o, attn_mask=future_mask(n).to(device))

        m2 = att_out2 + m1
        m2 = m2.permute(1,0,2)
        m2 = self.r_norm3(m2)

        l = m2 + self.ffn(m2)

        return l


def get_clones(module, N): #모듈 리스트
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class SAINT(nn.Module):
    def __init__(self, dim_model, num_en, num_de, heads_en, total_ex, total_in, heads_de, seq_len):
        super().__init__()

        self.num_en = num_en
        self.num_de = num_de

        self.encoder = get_clones( Encoder(n_in=total_ex, seq_len=seq_len, embed_dim=dim_model, nheads=heads_en) , num_en)
        self.decoder = get_clones( Decoder(n_in=total_in, seq_len=seq_len, embed_dim=dim_model, nheads=heads_de) , num_de)

        self.out = nn.Linear(in_features= dim_model , out_features=1)

    def forward(self, in_ex, in_in, diff):

        ## pass through each of the encoder blocks in sequence
        first_block = True
        for x in range(self.num_en):
            if x>=1:
                first_block = False
            in_ex = self.encoder[x](in_ex,diff, first_block=first_block)

        ## pass through each decoder blocks in sequence
        first_block = True
        for x in range(self.num_de):
            if x>=1:
                first_block = False
            in_in = self.decoder[x]( in_in , in_ex, first_block=first_block )

        ## Output layer
        in_in = torch.sigmoid( self.out( in_in ) )
        return in_in.squeeze(-1)


In [ ]:
def train_epoch(model, train_iterator, optim, criterion, device="cpu"):
    model.train()

    train_loss = []
    num_corrects = 0
    num_total = 0
    labels = []
    outs = []

    tbar = tqdm(train_iterator)
    for item in tbar:
        e = item[0].to(device).long()
        r = item[1].to(device).long()
        label = item[2].to(device).float()
        diff = item[3].to(device).long()


        # Zero the gradients in the optimizer
        optim.zero_grad()
        # The results of one forward pass
        output = model(e, r, diff)
        # Calculate the loss
        loss = criterion(output, torch.sigmoid(label))
        # Calculate the gradients with respect to the loss
        loss.backward()
        # Adjust the parameters to minimize the loss based on these gradients
        optim.step()
        # Add our loss to the list of losses
        train_loss.append(loss.item())

        output = output[:, -1]
        label = label[:, -1]
        pred = (output >= 0.5).long()

        num_corrects += (pred == label).sum().item()
        num_total += len(label)

        labels.extend(label.view(-1).data.cpu().numpy())
        outs.extend(output.view(-1).data.cpu().numpy())

        tbar.set_description('loss - {:.4f}'.format(loss))

    acc = num_corrects / num_total
    auc = roc_auc_score(labels, outs)
    loss = np.mean(train_loss)

    return loss, acc, auc

In [ ]:
def valid_epoch(model, valid_iterator, criterion, device="cpu"):
    model.eval()

    valid_loss = []
    num_corrects = 0
    num_total = 0
    labels = []
    outs = []
    preds = []

    #tbar = tqdm(valid_iterator)
    for item in valid_iterator: # tbar:
        e = item[0].to(device).long()
        r = item[1].to(device).long()
        label = item[2].to(device).float()
        diff = item[3].to(device).long()


        with torch.no_grad():
            output = model(e, r, diff)
        loss = criterion(output, torch.sigmoid(label))
        valid_loss.append(loss.item())

        output = output[:, -1] # (BS, 1)
        label = label[:, -1]
        pred = (output >= 0.6).long()


        num_corrects += (pred == label).sum().item()
        num_total += len(label)

        preds.extend(pred.view(-1).data.cpu().numpy())
        labels.extend(label.view(-1).data.cpu().numpy())
        outs.extend(output.view(-1).data.cpu().numpy())



    acc = num_corrects / num_total
    auc = roc_auc_score(labels, outs)
    print(labels)
    print(preds)
    print(outs)

    confusion_matrix = metrics.confusion_matrix(labels, preds)

    print('confusion matrix')
    print(confusion_matrix)



    loss = np.mean(valid_loss)

    return loss, acc, auc

In [ ]:
gc.collect()
NUM_QUESTIONS = 9985
MAX_SEQ = 100
BS = 64
NUM_DIFFS = 10
train_dataset = SAINTDataset(train_group, NUM_QUESTIONS, max_seq=MAX_SEQ)
train_dataloader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=8)

valid_dataset = SAINTDataset(valid_group, NUM_QUESTIONS, max_seq=MAX_SEQ, subset='valid')
valid_dataloader = DataLoader(valid_dataset, batch_size=BS, shuffle=False, num_workers=8)

In [ ]:
import copy
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SAINT(dim_model=128,
            num_en=2,
            num_de=2,
            heads_en=4,
            heads_de=4,
            total_ex=NUM_QUESTIONS,
            total_in=2,
            seq_len=100
            )

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCELoss()

model.to(device)
criterion.to(device)

BCELoss()

In [ ]:
gc.collect()
epochs = 30
history = []
auc_max = -np.inf

for epoch in range(1, epochs+1):
    train_loss, train_acc, train_auc = train_epoch(model, train_dataloader, optimizer, criterion, device)
    print(f'Epoch {epoch}, train_loss: {train_loss:5f}, train_acc: {train_acc:5f}, train_auc: {train_auc:5f}')
    valid_loss, valid_acc, valid_auc = valid_epoch(model, valid_dataloader, criterion, device)
    print(f'Epoch {epoch}, valid_loss: {valid_loss:5f}, valid_acc: {valid_acc:5f}, valid_auc: {valid_auc:5f}')

    lr = optimizer.param_groups[0]['lr']
    history.append({"epoch":epoch, "lr": lr, **{"train_auc": train_auc, "train_acc": train_acc}, **{"valid_auc": valid_auc, "valid_acc": valid_acc}})
    if valid_auc > auc_max:
        print("Epoch#%s, valid loss %.4f, Metric loss improved from %.4f to %.4f, saving model ..." % (epoch, valid_loss, auc_max, valid_auc))
        auc_max = valid_auc

loss - 0.6574: 100%|██████████| 409/409 [00:16<00:00, 24.77it/s]


Epoch 1, train_loss: 0.661329, train_acc: 0.713652, train_auc: 0.710023


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0,

  0%|          | 0/409 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
loss - 0.6607: 100%|██████████| 409/409 [00:17<00:00, 24.05it/s]

...

Epoch 10, train_loss: 0.656916, train_acc: 0.716406, train_auc: 0.842139



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0,

  0%|          | 0/409 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
loss - 0.6592:  28%|██▊       | 116/409 [00:05<00:14, 19.78it/s]


KeyboardInterrupt: ignored

### 인코더 diff 결과

In [ ]:
gc.collect()
epochs = 30
history = []
auc_max = -np.inf

for epoch in range(1, epochs+1):
    train_loss, train_acc, train_auc = train_epoch(model2, train_dataloader, optimizer, criterion, device)
    print(f'Epoch {epoch}, train_loss: {train_loss:5f}, train_acc: {train_acc:5f}, train_auc: {train_auc:5f}')
    valid_loss, valid_acc, valid_auc = valid_epoch(model2, valid_dataloader, criterion, device)
    print(f'Epoch {epoch}, valid_loss: {valid_loss:5f}, valid_acc: {valid_acc:5f}, valid_auc: {valid_auc:5f}')

    lr = optimizer.param_groups[0]['lr']
    history.append({"epoch":epoch, "lr": lr, **{"train_auc": train_auc, "train_acc": train_acc}, **{"valid_auc": valid_auc, "valid_acc": valid_acc}})
    if valid_auc > auc_max:
        print("Epoch#%s, valid loss %.4f, Metric loss improved from %.4f to %.4f, saving model ..." % (epoch, valid_loss, auc_max, valid_auc))
        auc_max = valid_auc

loss - 0.6529: 100%|██████████| 364/364 [00:16<00:00, 21.75it/s]


Epoch 1, train_loss: 0.660654, train_acc: 0.715251, train_auc: 0.703300


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 1, valid_loss: 0.659294, valid_acc: 0.707401, valid_auc: 0.762595
Epoch#1, valid loss 0.6593, Metric loss improved from -inf to 0.7626, saving model ...


  0%|          | 0/364 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
loss - 0.6616: 100%|██████████| 364/364 [00:14<00:00, 25.29it/s]


...

Epoch 29, valid_loss: 0.659039, valid_acc: 0.707401, valid_auc: 0.816900


  0%|          | 0/364 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
loss - 0.6666: 100%|██████████| 364/364 [00:14<00:00, 24.81it/s]


Epoch 30, train_loss: 0.654722, train_acc: 0.719167, train_auc: 0.892166


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 30, valid_loss: 0.659059, valid_acc: 0.707573, valid_auc: 0.817019


### Epoch7, valid loss 0.6580, Metric loss improved from 0.8406 to 0.8430

In [ ]:
gc.collect()
epochs = 30
history = []
auc_max = -np.inf

for epoch in range(1, epochs+1):
    train_loss, train_acc, train_auc = train_epoch(model2, train_dataloader, optimizer, criterion, device)
    print(f'Epoch {epoch}, train_loss: {train_loss:5f}, train_acc: {train_acc:5f}, train_auc: {train_auc:5f}')
    valid_loss, valid_acc, valid_auc = valid_epoch(model2, valid_dataloader, criterion, device)
    print(f'Epoch {epoch}, valid_loss: {valid_loss:5f}, valid_acc: {valid_acc:5f}, valid_auc: {valid_auc:5f}')

    lr = optimizer.param_groups[0]['lr']
    history.append({"epoch":epoch, "lr": lr, **{"train_auc": train_auc, "train_acc": train_acc}, **{"valid_auc": valid_auc, "valid_acc": valid_acc}})
    if valid_auc > auc_max:
        print("Epoch#%s, valid loss %.4f, Metric loss improved from %.4f to %.4f, saving model ..." % (epoch, valid_loss, auc_max, valid_auc))
        auc_max = valid_auc

loss - 0.6755: 100%|██████████| 364/364 [00:16<00:00, 21.78it/s]


Epoch 1, train_loss: 0.660508, train_acc: 0.713443, train_auc: 0.748591


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 1, valid_loss: 0.658032, valid_acc: 0.723408, valid_auc: 0.785647
Epoch#1, valid loss 0.6580, Metric loss improved from -inf to 0.7856, saving model ...


  0%|          | 0/364 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
loss - 0.6516: 100%|██████████| 364/364 [00:13<00:00, 26.34it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/free

...

Epoch 29, valid_loss: 0.658461, valid_acc: 0.721859, valid_auc: 0.804883


  0%|          | 0/364 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
loss - 0.6288: 100%|██████████| 364/364 [00:14<00:00, 25.63it/s]

Epoch 30, train_loss: 0.654735, train_acc: 0.715380, train_auc: 0.899616



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 30, valid_loss: 0.658628, valid_acc: 0.721859, valid_auc: 0.803465


In [ ]:
gc.collect()
epochs = 30
history = []
auc_max = -np.inf

for epoch in range(1, epochs+1):
    train_loss, train_acc, train_auc = train_epoch(model2, train_dataloader, optimizer, criterion, device)
    print(f'Epoch {epoch}, train_loss: {train_loss:5f}, train_acc: {train_acc:5f}, train_auc: {train_auc:5f}')
    valid_loss, valid_acc, valid_auc = valid_epoch(model2, valid_dataloader, criterion, device)
    print(f'Epoch {epoch}, valid_loss: {valid_loss:5f}, valid_acc: {valid_acc:5f}, valid_auc: {valid_auc:5f}')

    lr = optimizer.param_groups[0]['lr']
    history.append({"epoch":epoch, "lr": lr, **{"train_auc": train_auc, "train_acc": train_acc}, **{"valid_auc": valid_auc, "valid_acc": valid_acc}})
    if valid_auc > auc_max:
        print("Epoch#%s, valid loss %.4f, Metric loss improved from %.4f to %.4f, saving model ..." % (epoch, valid_loss, auc_max, valid_auc))
        auc_max = valid_auc

  0%|          | 0/361 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
loss - 0.6594: 100%|██████████| 361/361 [00:14<00:00, 25.18it/s]


Epoch 1, train_loss: 0.659565, train_acc: 0.716260, train_auc: 0.742201


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 1, valid_loss: 0.658256, valid_acc: 0.716237, valid_auc: 0.784549
Epoch#1, valid loss 0.6583, Metric loss improved from -inf to 0.7845, saving model ...


  0%|          | 0/361 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
loss - 0.6546: 100%|██████████| 361/361 [00:14<00:00, 24.43it/s]

...

Epoch 29, valid_loss: 0.658745, valid_acc: 0.715200, valid_auc: 0.796720


  0%|          | 0/361 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
loss - 0.6587: 100%|██████████| 361/361 [00:14<00:00, 24.76it/s]


Epoch 30, train_loss: 0.654396, train_acc: 0.718772, train_auc: 0.903014


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 30, valid_loss: 0.658948, valid_acc: 0.715200, valid_auc: 0.794338


In [ ]:
gc.collect()
epochs = 30
history = []
auc_max = -np.inf

for epoch in range(1, epochs+1):
    train_loss, train_acc, train_auc = train_epoch(model, train_dataloader, optimizer, criterion, device)
    print(f'Epoch {epoch}, train_loss: {train_loss:5f}, train_acc: {train_acc:5f}, train_auc: {train_auc:5f}')
    valid_loss, valid_acc, valid_auc = valid_epoch(model, valid_dataloader, criterion, device)
    print(f'Epoch {epoch}, valid_loss: {valid_loss:5f}, valid_acc: {valid_acc:5f}, valid_auc: {valid_auc:5f}')

    lr = optimizer.param_groups[0]['lr']
    history.append({"epoch":epoch, "lr": lr, **{"train_auc": train_auc, "train_acc": train_acc}, **{"valid_auc": valid_auc, "valid_acc": valid_acc}})
    if valid_auc > auc_max:
        print("Epoch#%s, valid loss %.4f, Metric loss improved from %.4f to %.4f, saving model ..." % (epoch, valid_loss, auc_max, valid_auc))
        auc_max = valid_auc

loss - 0.6501: 100%|██████████| 361/361 [00:14<00:00, 24.48it/s]

Epoch 1, train_loss: 0.660059, train_acc: 0.717712, train_auc: 0.747340



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 1, valid_loss: 0.658533, valid_acc: 0.707828, valid_auc: 0.790009
Epoch#1, valid loss 0.6585, Metric loss improved from -inf to 0.7900, saving model ...


  0%|          | 0/361 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
loss - 0.6549: 100%|██████████| 361/361 [00:14<00:00, 25.44it/s]


...

Epoch 28, train_loss: 0.654628, train_acc: 0.719661, train_auc: 0.885449


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 28, valid_loss: 0.658729, valid_acc: 0.707655, valid_auc: 0.815323


  0%|          | 0/361 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
loss - 0.6579:  33%|███▎      | 119/361 [00:05<00:11, 20.99it/s]


KeyboardInterrupt: ignored

In [ ]:
gc.collect()
epochs = 30
history = []
auc_max = -np.inf

for epoch in range(1, epochs+1):
    train_loss, train_acc, train_auc = train_epoch(model, train_dataloader, optimizer, criterion, device)
    print(f'Epoch {epoch}, train_loss: {train_loss:5f}, train_acc: {train_acc:5f}, train_auc: {train_auc:5f}')
    valid_loss, valid_acc, valid_auc = valid_epoch(model, valid_dataloader, criterion, device)
    print(f'Epoch {epoch}, valid_loss: {valid_loss:5f}, valid_acc: {valid_acc:5f}, valid_auc: {valid_auc:5f}')

    lr = optimizer.param_groups[0]['lr']
    history.append({"epoch":epoch, "lr": lr, **{"train_auc": train_auc, "train_acc": train_acc}, **{"valid_auc": valid_auc, "valid_acc": valid_acc}})
    if valid_auc > auc_max:
        print("Epoch#%s, valid loss %.4f, Metric loss improved from %.4f to %.4f, saving model ..." % (epoch, valid_loss, auc_max, valid_auc))
        auc_max = valid_auc

loss - 0.6644: 100%|██████████| 438/438 [00:20<00:00, 21.21it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 1, train_loss: 0.659503, train_acc: 0.716862, train_auc: 0.755001
Epoch 1, valid_loss: 0.655156, valid_acc: 0.707036, valid_auc: 0.777008
Epoch#1, valid loss 0.6552, Metric loss improved from -inf to 0.7770, saving model ...


  0%|          | 0/438 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
loss - 0.6498: 100%|██████████| 438/438 [00:17<00:00, 25.74it/s]


Epoch 2, train_loss: 0.657992, train_acc: 0.718255, train_auc: 0.782300


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

Epoch 29, valid_loss: 0.655563, valid_acc: 0.704729, valid_auc: 0.806414


  0%|          | 0/438 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
loss - 0.6551: 100%|██████████| 438/438 [00:17<00:00, 24.85it/s]


Epoch 30, train_loss: 0.654724, train_acc: 0.718147, train_auc: 0.893241


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 30, valid_loss: 0.655556, valid_acc: 0.704729, valid_auc: 0.799886


In [ ]:
class SAINT2Dataset(Dataset):
    def __init__(self, user_sequences, num_questions, subset='train', max_seq=100, min_seq=10):
        super(SAINT2Dataset, self).__init__()
        self.max_seq = max_seq
        self.num_questions = num_questions
        self.user_sequences = user_sequences
        self.subset = subset

        self.user_ids = []
        for user_id in user_sequences.index:
            q, _ = user_sequences[user_id]
            # if len(q) < min_seq:
            #     continue
            self.user_ids.append(user_id)

    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, index):
        user_id = self.user_ids[index]

        q_, qa_ = self.user_sequences[user_id]
        seq_len = len(q_)

        q = np.zeros(self.max_seq, dtype=int)
        qa = np.zeros(self.max_seq, dtype=int)
        #diff = np.zeros(self.max_seq, dtype=int)


        # If there are more questions answered than max_seq, take the last max_seq sequences
        if seq_len >= self.max_seq:
            q[:] = q_[-self.max_seq:]
            qa[:] = qa_[-self.max_seq:]
            #diff[:] = diff_[-self.max_seq:]

        # If not, map our user_sequences to the tail end of q and qa, the start will be padded with zeros
        else:
            q[-seq_len:] = q_
            qa[-seq_len:] = qa_
            #diff[-seq_len:] = diff_


        r = np.zeros(self.max_seq, dtype=int)   #shifted qa
        r[1:] = qa[:-1].copy()

        return q, r, qa#, diff


### diff X

In [ ]:
class FFN(nn.Module):
    def __init__(self, dim=128):
        super().__init__()
        self.layer1 = nn.Linear(dim, dim)
        self.layer2 = nn.Linear(dim, dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.layer2(   self.relu(   self.layer1(x)))


def future_mask(seq_length):    #다음 시점 마스킹
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)


class Encoder(nn.Module):
    def __init__(self, n_in, seq_len=100, embed_dim=128, nheads=4):
        super().__init__()

        #n_in: 입력 총개수

        self.seq_len = seq_len

        #self.part_embed = nn.Embedding(10, embed_dim)

        self.e_embed = nn.Embedding(n_in, embed_dim)
        self.e_pos_embed = nn.Embedding(seq_len, embed_dim)
        self.e_norm = nn.LayerNorm(embed_dim)

        self.e_multi_att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=nheads, dropout=0.2)
        self.m_norm = nn.LayerNorm(embed_dim)
        self.ffn = FFN(embed_dim)

    def forward(self, e, first_block=True):
        #e : q seq
        if first_block:
            e = self.e_embed(e)

        pos = torch.arange(self.seq_len).unsqueeze(0).to(device)
        e_pos = self.e_pos_embed(pos)
        e = e + e_pos
        e = self.e_norm(e)
        e = e.permute(1,0,2) #[bs, s_len, embed] => [s_len, bs, embed]
        n = e.shape[0]

        att_mask = future_mask(n).to(device)
        att_out, _ = self.e_multi_att(e, e, e, attn_mask=att_mask)
        m = e + att_out
        m = m.permute(1,0,2)

        o = m + self.ffn(self.m_norm(m))

        return o

class Decoder(nn.Module):
    def __init__(self, n_in, seq_len=100, embed_dim=128, nheads=4):
        super().__init__()
        self.seq_len = seq_len

        self.r_embed = nn.Embedding(n_in, embed_dim)    #r: 이전 시점 정답여부
        self.r_pos_embed = nn.Embedding(seq_len, embed_dim)
        self.r_norm = nn.LayerNorm(embed_dim)
        #self.diff_embed = nn.Embedding(NUM_DIFFS, embed_dim)


        self.r_multi_att1 = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=4, dropout=0.2)
        self.r_multi_att2 = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=4, dropout=0.2)
        self.ffn = FFN(embed_dim)

        self.r_norm1 = nn.LayerNorm(embed_dim)
        self.r_norm2 = nn.LayerNorm(embed_dim)
        self.r_norm3 = nn.LayerNorm(embed_dim)


    def forward(self, r, o,  first_block=True):

        if first_block:
            r = self.r_embed(r)
            #diff = self.diff_embed(diff)


            #r = r + diff

        pos = torch.arange(self.seq_len).unsqueeze(0).to(device)
        r_pos_embed = self.r_pos_embed(pos)
        r = r + r_pos_embed
        r = self.r_norm1(r)
        r = r.permute(1,0,2)
        n = r.shape[0]

        att_out1, _ = self.r_multi_att1(r, r, r, attn_mask=future_mask(n).to(device))
        m1 = r + att_out1

        o = o.permute(1,0,2)
        o = self.r_norm2(o)
        att_out2, _ = self.r_multi_att2(m1, o, o, attn_mask=future_mask(n).to(device))

        m2 = att_out2 + m1
        m2 = m2.permute(1,0,2)
        m2 = self.r_norm3(m2)

        l = m2 + self.ffn(m2)

        return l


def get_clones(module, N): #모듈 리스트
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class SAINT2(nn.Module):
    def __init__(self, dim_model, num_en, num_de, heads_en, total_ex, total_in, heads_de, seq_len):
        super().__init__()

        self.num_en = num_en
        self.num_de = num_de

        self.encoder = get_clones( Encoder(n_in=total_ex, seq_len=seq_len, embed_dim=dim_model, nheads=heads_en) , num_en)
        self.decoder = get_clones( Decoder(n_in=total_in, seq_len=seq_len, embed_dim=dim_model, nheads=heads_de) , num_de)

        self.out = nn.Linear(in_features= dim_model , out_features=1)

    def forward(self, in_ex, in_in):

        ## pass through each of the encoder blocks in sequence
        first_block = True
        for x in range(self.num_en):
            if x>=1:
                first_block = False
            in_ex = self.encoder[x](in_ex, first_block=first_block)

        ## pass through each decoder blocks in sequence
        first_block = True
        for x in range(self.num_de):
            if x>=1:
                first_block = False
            in_in = self.decoder[x]( in_in , in_ex, first_block=first_block )

        ## Output layer
        in_in = torch.sigmoid( self.out( in_in ) )
        return in_in.squeeze(-1)


In [ ]:
import pickle
with open('/content/drive/MyDrive/group_nm5.pkl', 'rb') as f:
    group = pickle.load(f)

In [ ]:
valid_group = group.sample(frac=0.2)
train_group = group.drop(valid_group.index).reset_index(drop=True)
valid_group.reset_index(drop=True, inplace=True)
train_group.shape, valid_group.shape

((23238,), (5810,))

In [ ]:
gc.collect()
NUM_QUESTIONS = 9985
MAX_SEQ = 100
BS = 64
NUM_DIFFS = 10
train_dataset = SAINT2Dataset(train_group, NUM_QUESTIONS, max_seq=MAX_SEQ)
train_dataloader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=8)

valid_dataset = SAINT2Dataset(valid_group, NUM_QUESTIONS, max_seq=MAX_SEQ, subset='valid')
valid_dataloader = DataLoader(valid_dataset, batch_size=BS, shuffle=False, num_workers=8)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
def train_epoch(model, train_iterator, optim, criterion, device="cpu"):
    model.train()

    train_loss = []
    num_corrects = 0
    num_total = 0
    labels = []
    outs = []

    tbar = tqdm(train_iterator)
    for item in tbar:
        e = item[0].to(device).long()
        r = item[1].to(device).long()
        label = item[2].to(device).float()
        #diff = item[3].to(device).long()


        # Zero the gradients in the optimizer
        optim.zero_grad()
        # The results of one forward pass
        output = model(e, r)
        # Calculate the loss
        loss = criterion(output, torch.sigmoid(label))
        # Calculate the gradients with respect to the loss
        loss.backward()
        # Adjust the parameters to minimize the loss based on these gradients
        optim.step()
        # Add our loss to the list of losses
        train_loss.append(loss.item())

        output = output[:, -1]
        label = label[:, -1]
        pred = (output >= 0.5).long()

        num_corrects += (pred == label).sum().item()
        num_total += len(label)

        labels.extend(label.view(-1).data.cpu().numpy())
        outs.extend(output.view(-1).data.cpu().numpy())

        tbar.set_description('loss - {:.4f}'.format(loss))

    acc = num_corrects / num_total
    auc = roc_auc_score(labels, outs)
    loss = np.mean(train_loss)

    return loss, acc, auc

In [ ]:
def valid_epoch(model, valid_iterator, criterion, device="cpu"):
    model.eval()

    valid_loss = []
    num_corrects = 0
    num_total = 0
    labels = []
    outs = []

    #tbar = tqdm(valid_iterator)
    for item in valid_iterator: # tbar:
        e = item[0].to(device).long()
        r = item[1].to(device).long()
        label = item[2].to(device).float()
        #diff = item[3].to(device).long()


        with torch.no_grad():
            output = model(e, r)
        loss = criterion(output, torch.sigmoid(label))
        valid_loss.append(loss.item())

        output = output[:, -1] # (BS, 1)
        label = label[:, -1]
        pred = (output >= 0.5).long()

        num_corrects += (pred == label).sum().item()
        num_total += len(label)

        labels.extend(label.view(-1).data.cpu().numpy())
        outs.extend(output.view(-1).data.cpu().numpy())

    acc = num_corrects / num_total
    auc = roc_auc_score(labels, outs)
    loss = np.mean(valid_loss)

    return loss, acc, auc

In [ ]:
import copy
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SAINT2(dim_model=128,
            num_en=2,
            num_de=2,
            heads_en=4,
            heads_de=4,
            total_ex=NUM_QUESTIONS,
            total_in=2,
            seq_len=100
            )

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCELoss()

model.to(device)
criterion.to(device)

BCELoss()

In [ ]:
gc.collect()
epochs = 30
history = []
auc_max = -np.inf

for epoch in range(1, epochs+1):
    train_loss, train_acc, train_auc = train_epoch(model, train_dataloader, optimizer, criterion, device)
    print(f'Epoch {epoch}, train_loss: {train_loss:5f}, train_acc: {train_acc:5f}, train_auc: {train_auc:5f}')
    valid_loss, valid_acc, valid_auc = valid_epoch(model, valid_dataloader, criterion, device)
    print(f'Epoch {epoch}, valid_loss: {valid_loss:5f}, valid_acc: {valid_acc:5f}, valid_auc: {valid_auc:5f}')

    lr = optimizer.param_groups[0]['lr']
    history.append({"epoch":epoch, "lr": lr, **{"train_auc": train_auc, "train_acc": train_acc}, **{"valid_auc": valid_auc, "valid_acc": valid_acc}})
    if valid_auc > auc_max:
        print("Epoch#%s, valid loss %.4f, Metric loss improved from %.4f to %.4f, saving model ..." % (epoch, valid_loss, auc_max, valid_auc))
        auc_max = valid_auc

loss - 0.6605: 100%|██████████| 364/364 [00:13<00:00, 26.01it/s]


Epoch 1, train_loss: 0.661289, train_acc: 0.716025, train_auc: 0.709047


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 1, valid_loss: 0.659103, valid_acc: 0.714630, valid_auc: 0.751554
Epoch#1, valid loss 0.6591, Metric loss improved from -inf to 0.7516, saving model ...


  0%|          | 0/364 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
loss - 0.6596: 100%|██████████| 364/364 [00:13<00:00, 26.30it/s]


...

Epoch 13, train_loss: 0.657646, train_acc: 0.717144, train_auc: 0.803788


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 13, valid_loss: 0.658283, valid_acc: 0.714802, valid_auc: 0.789826


  0%|          | 0/364 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
loss - 0.6591:  92%|█████████▏| 336/364 [00:12<00:01, 26.64it/s]


KeyboardInterrupt: ignored